# Cricket Biomechanics Analysis Demo

This notebook demonstrates the key components of the cricket straight drive biomechanics analysis system. It shows how to:

1. Extract pose data from a cricket batting video
2. Calculate joint angles and other biomechanical features
3. Visualize the results
4. Prepare data for the LSTM model

**Note:** For Sprint 1, this is a demonstration notebook showing the planned workflow. In future sprints, we will incorporate actual data and complete the implementation.

In [ ]:
# Import necessary libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pathlib import Path

# Add the src directory to the path
sys.path.append('../')

# Import our modules
from src.data_collection.pose_extractor import PoseExtractor
from src.feature_engineering.joint_angles import JointAngleCalculator
from src.visualization.pose_visualizer import PoseVisualizer
from src.model.lstm_model import StraightDriveClassifier, StraightDriveLSTM

## 1. Configuration

First, let's set up the paths and load the configuration file.

In [ ]:
# Set up paths
config_path = "../configs/config.yaml"
data_dir = "../data"
raw_dir = os.path.join(data_dir, "raw")
processed_dir = os.path.join(data_dir, "processed")

# Create directories if they don't exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)

# Load configuration
import yaml

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded successfully.")
print(f"Using {config['pose']['model']} for pose estimation.")
print(f"Model type: {config['model']['type']}")
print(f"Sequence length: {config['model']['sequence_length']} frames")

## 2. Data Collection

This section demonstrates how to extract pose data from a cricket batting video. In a real implementation, you would provide your own video file.

In [ ]:
# Initialize the pose extractor
pose_extractor = PoseExtractor(config_path)

# Example (commented out as we don't have a real video file yet)
# video_path = os.path.join(raw_dir, "sample_straight_drive.mp4")
# pose_data = pose_extractor.process_video(video_path, output_dir=processed_dir, visualize=True)

# For demonstration, we'll create a sample pose data structure
def create_sample_pose_data():
    """Create sample pose data for demonstration"""
    # Sample landmarks for 10 frames
    frames = []
    for i in range(10):
        # Create 33 landmarks (MediaPipe standard)
        landmarks = []
        for j in range(33):
            # Add some simulated movement
            landmarks.append({
                'x': 0.5 + 0.1 * np.sin(i * 0.1 + j * 0.05),
                'y': 0.5 + 0.1 * np.cos(i * 0.1 + j * 0.05),
                'z': 0.1 * np.sin(i * 0.1),
                'visibility': 0.9
            })
        
        frames.append({
            'frame_idx': i,
            'timestamp': i / 30.0,  # Assuming 30 FPS
            'landmarks': landmarks
        })
    
    pose_data = {
        'video_name': 'sample_straight_drive.mp4',
        'fps': 30.0,
        'frame_count': 10,
        'frames': frames
    }
    
    return pose_data

# Create sample data
sample_pose_data = create_sample_pose_data()

# Save sample data
sample_pose_path = os.path.join(processed_dir, "sample_pose.json")
with open(sample_pose_path, 'w') as f:
    json.dump(sample_pose_data, f, indent=2)

print(f"Generated sample pose data with {len(sample_pose_data['frames'])} frames.")
print(f"Each frame has {len(sample_pose_data['frames'][0]['landmarks'])} landmarks.")

## 3. Feature Engineering

Now, let's calculate joint angles and other biomechanical features from the pose data.

In [ ]:
# Initialize the joint angle calculator
angle_calculator = JointAngleCalculator()

# Process the sample pose data
df = angle_calculator.process_pose_data(sample_pose_path)
# Calculate dynamic features (velocities, accelerations)
df_with_dynamics = angle_calculator.calculate_dynamic_features(df)

# Save the features
features_path = os.path.join(processed_dir, "sample_features.csv")
df_with_dynamics.to_csv(features_path, index=False)

# Show the first few rows
print("Generated biomechanical features:")
df_with_dynamics.head()

## 4. Visualization

Let's visualize the joint angles we calculated.

In [ ]:
# Initialize the pose visualizer
visualizer = PoseVisualizer(config_path)

# Get angle columns (for demonstration)
angle_columns = [col for col in df_with_dynamics.columns if 'angle' in col]

# Create a time series plot
fig = visualizer.create_angle_time_series_plot(
    df_with_dynamics,
    angle_columns,
    os.path.join(processed_dir, "angle_plot.png")
)

# Display the plot
plt.figure(figsize=(12, 6))
for col in angle_columns:
    plt.plot(df_with_dynamics['timestamp'], df_with_dynamics[col], label=col)
plt.xlabel('Time (s)')
plt.ylabel('Angle (degrees)')
plt.title('Joint Angles During Cricket Straight Drive')
plt.legend()
plt.grid(True)
plt.show()

## 5. Preparing Data for the LSTM Model

Now, let's prepare the data for the LSTM model. 

In [ ]:
# Initialize the classifier
classifier = StraightDriveClassifier(config_path)

# For demonstration, let's add a dummy label column
df_with_dynamics['technique_label'] = 1  # 1 = correct, 0 = incorrect

# Prepare sequences for the LSTM model
seq_length = config['model']['sequence_length']
sequences, labels = classifier.prepare_sequences(
    df_with_dynamics,
    label_col='technique_label',
    seq_length=min(seq_length, len(df_with_dynamics) - 1)  # Ensure we have enough data
)

print(f"Prepared {len(sequences)} sequences with shape {sequences.shape}")
print(f"Labels shape: {labels.shape}")

## 6. Model Architecture

Here, we'll demonstrate the LSTM model architecture. In future sprints, we'll implement the actual training.

In [ ]:
# Build model
input_size = sequences.shape[2]  # Number of features
num_classes = len(np.unique(labels))

model = classifier.build_model(input_size, num_classes)

# Print model summary
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("LSTM Model Architecture:")
print(f"- Input size: {input_size}")
print(f"- Hidden size: {config['model']['hidden_units']}")
print(f"- Output classes: {num_classes}")
print(f"- Total trainable parameters: {count_parameters(model)}")

## 7. Next Steps

In future sprints, we will:

1. Collect and label real cricket batting videos
2. Extract pose data using MediaPipe
3. Engineer comprehensive biomechanical features
4. Train and evaluate the LSTM model
5. Implement real-time feedback mechanisms
6. Create a user interface for coaches and players

The foundation laid in Sprint 1 provides the structure for all these future developments.